In [1]:
import sys
import pandas as pd
from sqlalchemy import create_engine

In [9]:
def load_data(messages_filepath, categories_filepath):
    '''
       INPUT:
           messages_filepath (str): messages csv files path
           categories_filepath (str): categories csv file path
       OUTPUT:
           df: dataframe having messages and cateries details joined
       DESCRIPTION:
               read messages csv file as messages dataframe and
               categories csv file as categories dataframe
               merge both the dataframes as df applying inner join on ['id'] column
    '''

    df_messages = pd.read_csv(messages_filepath, encoding='latin-1')
    df_categories = pd.read_csv(categories_filepath, encoding='latin-1')

    # merge datasets
    df = pd.merge(df_messages, df_categories, how='inner', on='id')
    return df

In [14]:
def clean_data(df):
    '''
       INPUT:
          The function takes the dataframe as merges from 'load_data' and re-creates a columns from the data
          while dropping the category column.
          arg: dataframe
       OUTPUT:
           df: dataframe having messages and cateries details
    '''

   # create a dataframe of the each of the category type
    categories = df.categories.str.split(';', expand=True)

    # select the first row of the categories in the dataframe
    row = categories.iloc[0, :]

    # convert the row cells to columns using lambda expression.
    cols = row.apply(lambda x: x[:-2])

    # bind new columns to the `categories` dataframe.
    categories.columns = cols

    # convert category values to numbers 0 or 1
    for column in categories:
        # set each value to be the last character of the string
        categories[column] = categories[column].apply(lambda x: x[-1])

        # convert column from string to numeric
        categories[column] = pd.to_numeric(categories[column])

    df = df.drop('categories', axis=1)  # drop the original categories column from df
    df = pd.concat([df, categories], axis=1)

    df = df.drop_duplicates()  # drop the duplicates

    return df

In [11]:
def save_data(df, database_filename):
    '''
    INPUT:
        cleansed dataframe having messages and their belonging categories details
    OUTPUT: 
        database having Messages table
    DESCRIPTION:
        Insert dataframe into sql table<DisasterMessages> in database file to be used as input   
    '''
    table = 'DisasterMessages'

    engine = create_engine('sqlite:///{}'.format(database_filename))
    

    df.to_sql(name=table, con=engine,if_exists='replace', chunksize=10, index=False) 
    


In [12]:
def main():
        df = load_data('./data/disaster_messages.csv','./data/disaster_categories.csv')

        print('Cleaning data...')
        df = clean_data(df)
        print('Saving data...')
        save_data(df, 'DisasterResponse.db')
        
        print('Cleaned data saved to database!')
    


In [15]:
main()

Cleaning data...
Saving data...
Cleaned data saved to database!


In [ ]:
python process_data.py disaster_messages.csv disaster_categories.csv DisasterResponse.db